本文主要是学习笔记，一方面是为了加强理解，感觉在做笔记过程中理解起来更简单，另一方面为了加强记忆，建立大脑关于‘隐马尔可夫模型’的神经网络，最后当然是为了后面查看方便

# 模型场景

按照下面的方式抽球，产生一个球的颜色的观测序列：
- (1)开始，从4个盒子里以等概率随机选取一个盒子，从这个盒子里随机抽出一个球，记录其颜色，然后放回
- (2)然后，从当前盒子随机转移到下一个盒子，规则是：如果当前盒子是盒子1，那么下一个盒子一定是盒子2，如果当前盒子是盒子2或3，那么分别以概率0.4和0.6转移到左边或右边的盒子，如果当前是盒子4，那么各以0.5的概率停留在盒子4或转移到盒子3
- (3)确定转移的盒子后，再从这个盒子里随机抽出一个球，记录其颜色，放回
- (4)如此下去，重复进行5次，得到一个球的颜色的观测序列：$$O = (红，红，白，白，红)$$

在这个过程中，观察者只能观测到球的颜色的序列，观测不到球是从哪个盒子取出的，即观测不到盒子的序列

# 隐马尔可夫模型三要素

上面的例子是一个典型的<b>隐马尔可夫模型</b>。有两个随机序列，一个是盒子的序列(状态序列)，一个是球的颜色的观测序列，前者是隐藏的，只有后者是可观测的。

隐马尔可夫模型有三要素，表示为$$\lambda = (A, B, \pi)$$
注：A为状态转移矩阵，B为观测概率分布矩阵，$\pi 为初始状态概率向量$

通过上面的例子，来分别计算下A,B和$\pi$的值

<b>状态转移概率分布矩阵：</b>
$$ A = 
 \left[
 \begin{matrix}
   0   & 1 &   0 & 0 \\
   0.4 & 0 & 0.4 & 0 \\
   0 & 0.4 &   0 & 0.6 \\
   0 &   0 & 0.5 & 0.5
  \end{matrix}
  \right]
$$
$A[ij]$表示从状态i转移到状态j的概率

<b>观测概率分布矩阵：</b>

$$ B = 
 \left[
 \begin{matrix}
   0.5 & 0.5 \\
   0.3 & 0.7 \\
   0.6 & 0.4 \\
   0.8 & 0.2
  \end{matrix}
  \right]
$$
$B[i0]$表示盒子i中取出红球的概率，$B[i1]$表示盒子i中取出白球的概率

<b>初始概率分布：</b>
$$\pi = (0.25,0.25,0.25,0.25)$$

# 隐马尔可夫模型的三个基本问题

(1) 概率计算问题

给定模型$\lambda = (A,B,\pi)$和观测序列$O = (o_1,o_2,...,0_T)$,计算在模型$\lambda$下观测模型出现的概率$P(O|\lambda)$

(2) 学习问题

已知观测序列$O = (o_1,o_2,...,0_T)$,估计模型$\lambda = (A,B,\pi)$参数，使得在该模型下观测序列概率$P(O|\lambda)$最大，用极大似然估计的方法估计参数

(3) 预测问题，也称为解码问题

已知模型$\lambda = (A,B,\pi)$和观测序列$O = (o_1,o_2,...,0_T)$，求对给定观测序列条件概率$P(O|\lambda)$最大的状态序列$I = (i_1,i_2,...,i_T)$。即给定观测序列，求最有可能的对应的状态序列

下面分别介绍针对不同问题的解决算法

# 概率计算算法

## 问题描述

给定模型$\lambda = (A,B,\pi)$和观测序列$O = (o_1,o_2,...,0_T)$,计算在模型$\lambda$下观测模型出现的概率$P(O|\lambda)$

## 前向算法

<b>(1) 计算状态t1下观测为红球的情况,注：序列和矩阵索引都从1开始</b>

第一次从盒子1选择红球的情况：
$$a_1(1) = \pi_1 B_1(o_1) = 0.25 * 0.5 = 0.125$$
第一次从盒子2选择红球的情况：
$$a_1(2) = \pi_2 B_2(o_1) = 0.25 * 0.3 = 0.075$$
第一次从盒子3选择红球的情况：
$$a_1(3) = \pi_3 B_3(o_1) = 0.25 * 0.6 = 0.15$$
第一次从盒子4选择红球的情况：
$$a_1(4) = \pi_4 B_4(o_1) = 0.25 * 0.8 = 0.20$$

<b>(2) 计算状态t2下观测为红球的情况，及第二次选择为红球的情况</b>

第二次从盒子1选择红球的情况：
$$a_2(1) = a_1(1)A_{11}B_1(o_2) + a_1(2)A_{21}B_1(o_2) + + a_1(3)A_{31}B_1(o_2) + + a_1(4)A_{41}B_1(o_2)$$
第二次从盒子2选择红球的情况：
$$a_2(2) = a_1(1)A_{12}B_2(o_2) + a_1(2)A_{22}B_2(o_2) + + a_1(3)A_{32}B_2(o_2) + + a_1(4)A_{42}B_2(o_2)$$
第二次从盒子3选择红球的情况：
$$a_2(3) = a_1(1)A_{13}B_3(o_2) + a_1(2)A_{23}B_3(o_2) + + a_1(3)A_{33}B_3(o_2) + + a_1(4)A_{43}B_3(o_2)$$
第二次从盒子4选择红球的情况：
$$a_2(4) = a_1(1)A_{14}B_4(o_2) + a_1(2)A_{24}B_4(o_2) + + a_1(3)A_{34}B_4(o_2) + + a_1(4)A_{44}B_4(o_2)$$

...

<b>通过上述规律我们得到公式:</b>

(1) 初值
$$a_1(i) = \pi(i)B_i(o_1)$$
(2) 递推
$$a_{t+1}(i) = [\sum_{j=1}^N a_t(j)A_{ji}]B_i(o_{t+1}) $$
(3) 终止
$$P(O|\lambda) = \sum_{i=1}^N a_T(i)$$

## 后向算法

顾名思义，后向算法就是根据t时刻的观测序列概率算出t-1时刻观测序列的概率

令在t时刻状态为$q_i$的条件下，从t+1到T的观测序列的概率为$\beta_t(i)$,则$$\beta_t(i) = P(o_{t+1}，o_{t+2},...,o_T|i_t=q_i,\lambda)$$

<b>要特别注意$\beta_t(i)$的定义，后面才能很好的理解</b>

(1) 对最终时刻的所有状态$q_i$规定$$\beta_T(i) = 1$$

(2) $$\beta_t(i) = \sum_{j=1}^N a_{ij}b_j(0_{t+1})\beta_{t+1}(j)$$

(3) $$P(O|\lambda) = \sum_{i=1}^N\pi_ib_i(o_1)\beta_1(i)$$

# 学习算法

## 问题描述

已知观测序列$O = (o_1,o_2,...,0_T)$,估计模型$\lambda = (A,B,\pi)$参数，使得在该模型下观测序列概率$P(O|\lambda)$最大，用极大似然估计的方法估计参数

隐马尔可夫模型的学习，根据训练数据集是包括观测序列和对应的状态序列还是只有观测序列，可以分别由监督学习与无监督学习实现

对于监督学习，由于数据集包含了观测序列和对应的状态序列，这样就可以直接根据利用数据集预估模型参数

对于非监督学习，可以使用EM算对隐参数进行学习。EM算法参考附录

# 预测算法

## 问题描述

已知模型$\lambda = (A,B,\pi)$和观测序列$O = (o_1,o_2,...,0_T)$，求对给定观测序列条件概率$P(O|\lambda)$最大的状态序列$I = (i_1,i_2,...,i_T)$。即给定观测序列，求最有可能的对应的状态序列

## 维特比算法

维特比算法实际是用动态规划解隐马尔可夫模型预测问题，即用动态规划求概率最大路径

定义两个变量：

$\delta_t(i)$表示在时刻t状态为i的所有单个路径中的最大概率值
$$\delta_t(i) = max P(i_t=i,i_{t-1},...,i_1,o_t,...,o_1|\lambda), i = 1,2,...,N$$

$\psi_t(i)$表示在时刻t状态为i的所有单个路径中概率最大的路径的第t-1个节点
$$\psi_t(i) = arg max_{1<=j<=N} [\delta_{t-1}(j)a_{ji}],i = 1,2,...,N$$

(1) 初始化$$\delta_1(i) = \pi_ib_i(o_1)$$
$$\psi_1(i) = 0$$

(2) 递推，对t=2,3,...,T
$$\delta_t(i) = max_{1<=j<=N} [\delta_{t-1}(j)a_{ji}]b_i(o_t)$$
$$\psi_t(i) = arg max_{1<=j<=N} [\delta_{t-1}(j)a_{ji}]$$

(3) 终止
$$P^* = max_{1<=i<=N}\delta_T(i)$$
$$i_T^* = arg max_{1<=i<=N} [\delta_T(i)]$$

# 附：EM算法

## EM算法定义

输入：观测变量数据X，隐变量数据Z,联合分布$P(X,Z|\theta)$,也称为完全数据，这样更好理解点

输出：模型参数$\theta$

(1)选择初始模型参数$\theta^{(0)}$，开始迭代

(2)E步：记$\theta^{i}$为第i次迭代参数$\theta$的估计值，计算在第i次迭代的期望$$Q(\theta,\theta^{(i)}) = E(logP(x,z|\theta)|x,\theta^{(i)}))=\int_zlogp(x,z|\theta)p(z|\theta^{(i)})$$
(3)M步：求使$\theta^{(i+1)} = Q(\theta,\theta^{(i)})的最大值$

(4)重复第(2)步和第(3)步

## EM算法几点说明

(1)参数的初值可以任意选择，但需注意EM算法对初始值是敏感的

(2)E步求$Q(\theta,\theta^{(i)})$,Q函数中的Z是为隐变量，X是观测数据，$Q(\theta,\theta^{(i)})$中的第一个变元表示要极大化的参数，第二个变元表示参数的当前估计值，每次迭代实际在求Q的极大值

(3)给出停止迭代的条件，一般是对较小的正数$\xi_i,\xi_2$,若满足$||\theta^{(i+1)} - \theta^{(i)} < \xi_i||或||Q(\theta^{(i+1)},\theta^{(i)})-Q(\theta^{(i)},\theta^{(i)})|| < \xi_2$

## EM算法推导

$$L(\theta)= argmaxlogP(x|\theta) = argmaxlog\int_zp(x,z|\theta)dz$$
$$L(\theta) = argmaxlog\int_z\frac{p(x,z|\theta)}{p(z|\theta^{(i)})}p(z|\theta^{(i)})dz$$
由于log函数为凹函数，则$$L(\theta) \geq \int_zlog\frac{p(x,z|\theta)}{p(z|\theta^{(i)})}p(z|\theta^{(i)})dz$$
$$L(\theta) \geq \int_zlogp(x,z|\theta)p(z|\theta^{(i)})dz - \int_zlog(p(z|\theta^{(i)}))p(z|\theta^{(i)})dz$$
由于减式后面与模型参数$\theta$无关，$P(z|\theta^{(i)})是已知的$，所以只需关注减式前面的式子，令$$Q(\theta,\theta^{(i)})=\int_zlogp(x,z|\theta)p(z|\theta^{(i)})$$
和算法定义中的步骤(2)相同，将原L的优化问题转换为求原问题下界$Q(\theta,\theta^{(i)})$的最大值
因此，任何可以使$Q(\theta,\theta^{(i)})$增大的$\theta$都可以使$L(\theta)$增大,为了使$L(\theta)$有尽可能的增长，选择使$Q(\theta,\theta^{(i)})$达到最大，即$$\theta^{(i+1)} = argmaxQ(\theta,\theta^{(i)})$$

## EM算法收敛性

<b>定理1</b>：$设P(x|\theta)为观测数据的似然函数，\theta^{(i)}为EM算法得到的参数估计序列，P(x|\theta^{(i)})为对应的似然函数序列，则P(x|\theta^{(i)})单调递增$
<b>定理2</b>：$设L(\theta) = logP(x|\theta)为观测数据的似然函数，\theta^{(i)}为EM算法得到的参数估计序列，L(\theta^{(i)})为对应的似然函数序列$

(1)$如果P(x|\theta)有上界，则L(\theta^{(i)})收敛到某一值L^*$
(2)$在函数Q(\theta,\theta^{(i)})与L(\theta)满足一定条件下，由EM算法得到的参数估计序列\theta^{(i)}的收敛值\theta^*是L(\theta)的稳定值$

<b>以上为EM算法的'官方'说明，若不理解可以参考博客https://www.jianshu.com/p/1121509ac1dc<b>

最后针对隐马尔可夫模型抛出抛出两个问题：

(1) 如何对中文分词问题用隐马尔可夫模型进行建模和训练？

(2) 最大熵马尔可夫模型为什么会产生标注偏置问题？如何解决？

<b>参考资料：</b>
李航老师的《统计学习方法》